## ResNet
- 주요 아이디어 : 
    - (1) 잔차 학습 Block : 합성곱 연산을 통과한 결과와 입력으로 들어온 결과 두가지를 더해 다음 레이어에 전달하는 것
    - (2) Bottlneck 방식 : 1x1 conv를 활용한 Bottleneck 방식

- 잔차 Module 2가지
    - (1) 가로 세로 해상도가 바뀌지 않는 경우
    - (2) 가로 세로 해상도가 다운샘플링 되는 경우
    


## ResNet50 Architecture

![rsenet50](img/rsenet50.png)

## ResNet Block
- (1) 가로 세로 해상도 그대로인 경우
- (2) 가로 세로 해상도 다운샘플링 되는 경우

In [21]:
import torch
import torch.nn as nn

class BottleNeck(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim, act_fn, down=False):
        super(BottleNeck, self).__init__()
        self.act_fn = act_fn
        self.down = down # 다운샘플링 여부
        
        # down smapling 용
        if self.down:
            self.layer = nn.Sequential(
                self.conv_block_1(in_dim, mid_dim, act_fn, 2), # stride 2
                self.conv_block_3(mid_dim, mid_dim, act_fn),
                self.conv_block_1(mid_dim, out_dim, act_fn),
            )
            self.downsample = nn.Conv2d(in_dim, out_dim, 1, 2) # stride 2 
            
        else:
            self.layer = nn.Sequential(
                self.conv_block_1(in_dim, mid_dim, act_fn, 1),
                self.conv_block_3(mid_dim, mid_dim, act_fn),
                self.conv_block_1(mid_dim, out_dim, act_fn)
            )
            self.dim_equalizer = nn.Conv2d(in_dim, out_dim, kernel_size=1) # 차원이 안맞을 경우 1x1로 맞춰줌
            
    def forward(self, x):
        # downsample이 있을 경우 downsampling
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = out + downsample

        # channel size가 안맞을 때 1x1 로 맞춰줌
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
            out = out + x
        return out

    # conv 1x1 block
    def conv_block_1(self, i_dim, o_dim, act_fn, stride = 1):
        model = nn.Sequential(
            nn.Conv2d(i_dim, o_dim, kernel_size=1, stride=stride),
            act_fn,
        )
        return model

    # conv 3x3 block
    def conv_block_3(self, i_dim, o_dim, act_fn):
        model = nn.Sequential(
            nn.Conv2d(i_dim, o_dim, kernel_size = 3, stride=1, padding=1),
            act_fn,
        )

In [22]:
# 3개, 4개, 6개, 3개
class ResNet50(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(ResNet50, self).__init__()
        self.act_fn = nn.ReLU()
        
        # Init Layer 1
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3, base_dim, 7, 2, 3), # in_dim, out_dim, kernel size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3,2,1) # 3x3 maxpooling, strides 2, padding 1
        )
        
        # BottleNeck 1 - 3개짜리
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim, base_dim, base_dim*4, self.act_fn), # ex) 64 64 256
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn), # 256 -> 128, 128, 512
            BottleNeck(base_dim*4, base_dim, base_dim*4, self.act_fn, down = True), 
        )
        
        # BottleNeck 1 - 4개짜리
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn),
            BottleNeck(base_dim*8, base_dim*2, base_dim*8, self.act_fn, down = True),
        )

        # BottleNeck 1 - 6개짜리
        self.layer_4 = nn.Sequential(
            BottleNeck(base_dim*8, base_dim*4, base_dim*16, self.act_fn), # 512 -> 256 , 256, 1024
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn),
            BottleNeck(base_dim*16, base_dim*4, base_dim*16, self.act_fn, down = True),
        )
        
        # BottleNeck 1 - 3개짜리
        self.layer_5 = nn.Sequential(
            BottleNeck(base_dim*16, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
            BottleNeck(base_dim*32, base_dim*8, base_dim*32, self.act_fn),
        )
        self.avgpool = nn.AvgPool2d(7,1)
        self.fc_layer = nn.Linear(base_dim*32, num_classes)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(x)
        out = self.layer_3(x)
        out = self.layer_4(x)
        out = self.layer_5(x)
        out = self.avgpool(out)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

In [23]:
if __name__ == "__main__":
    model = ResNet50(base_dim=64)
    print(model)

ResNet50(
  (act_fn): ReLU()
  (layer_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): BottleNeck(
      (act_fn): ReLU()
      (layer): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU()
        )
        (1): None
        (2): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU()
        )
      )
      (dim_equalizer): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BottleNeck(
      (act_fn): ReLU()
      (layer): Sequential(
        (0): Sequential(
          (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU()
        )
        (1): None
        (2): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
          (1): ReL